In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
plt.style.use('dark_background')

# Diferencias finitas

Primera derivada numérica central:

\begin{equation}
\frac{df}{dx} \approx \frac {f(x_0+h)-f(x_0-h)} {2h}
\end{equation}

Segunda derivada numérica central:

\begin{equation}
\frac{d^2f}{dx^2}\approx \frac {f(x_0+h)-2 f(x_0)+f(x_0-h)} {h^2}
\end{equation}

## Ecuación de onda 1D (extremos fijos)
Vamos a resolver la ecuación de onda en una dimensión:

\begin{equation}
 \frac{\partial^2 u}{\partial t^2} =
{c^2} \frac{\partial^2 u}{\partial x^2} 
\end{equation}

\begin{equation}
u = u(t,x)
\end{equation}

En términos de diferencias finitas queda

\begin{equation}
\frac{u^{n+1}_{i}-2u^{n}_{i}+u^{n-1}_{i}}{(\Delta t)^2}  = c^2\frac{u^{n}_{i+1}-2u^{n}_{i}+u^{n}_{i-1}}{(\Delta x)^2}
\end{equation}

\begin{equation}
u^{n+1}_{i}-2u^{n}_{i}+u^{n-1}_{i} = r(u^{n}_{i+1}-2u^{n}_{i}+u^{n}_{i-1})
\end{equation}

donde $r = (\Delta t)^2 \,c^2/(\Delta x)^2$. 

\begin{equation}
u^{n+1}_{i} = 2u^{n}_{i}-u^{n-1}_{i}+ r(u^{n}_{i+1}-2u^{n}_{i}+u^{n}_{i-1})
\end{equation}

\begin{equation}
u^{n+1}_{i} = 2(1-r)u^{n}_{i}-u^{n-1}_{i}+ ru^{n}_{i+1}+ru^{n}_{i-1}
\end{equation}

\begin{equation}
\boxed{u^{n+1}_{i} = au^{n}_{i}-u^{n-1}_{i}+ r(u^{n}_{i+1}+u^{n}_{i-1})}
\end{equation}

donde $a=2(1-r)$. Es necesario contar con condiciones de frontera (por ejemplo: $u(t, 0) = l_0, u(t, L) = l_L$) y condiciones iniciales para la posición (ejemplo: $u(0,x) = f(x)$) y la velocidad (como esta: $du(0,x)/dx = V_0(x)$)

para encontrar $u^{0-1}_i$ se puede hacer uso de definición de primera derivada numérica sobre la condición de velocidad inicial:
\begin{equation}
\frac{\partial u(0,x) }{ \partial t} = V_0(x)
\end{equation}

\begin{equation}
\frac{u^{1}_{i}-u^{-1}_{i}}{2\Delta t} = V^{0}_{i}
\end{equation}

\begin{equation}
u^{-1}_{i} = u^{1}_{i} - 2V^{0}_{i}\Delta t
\end{equation}

Al sustituir en la ecuación de onda discretizada para $t = 0$:

\begin{equation}
u^{1}_{i} = au^{0}_{i}-(u^{1}_{i} - 2V^{0}_{i}\Delta t)+ r(u^{0}_{i+1}+u^{0}_{i-1})
\end{equation}

\begin{equation}
\boxed{u^{1}_{i} = \frac{a}{2}u^{0}_{i}+ \frac{r}{2}(u^{0}_{i+1}+u^{0}_{i-1}) + V^{0}_{i}\Delta t}
\end{equation}

Para que la solución sea estable se requiere que $a$ sea mayor que cero, lo que implica

\begin{equation}
0< 2\left( 1- \frac{c^2(\Delta t)^2}{(\Delta x)^2}\right)
\end{equation}

despejando $\Delta t$

\begin{equation}
\Delta t < \Delta x / c
\end{equation}

**Ejemplo con extremos fijos:**



In [ ]:
dx = 0.02   
L = 1.0
t_max = 1.5
c = 1.8

# condiciones de frontera 
l0 = 0
lL = 0

# condiciones iniciales
def pos_ini(x):
  sig = 0.1
  return 1.0/np.sqrt(sig)*np.exp(-((x-0.3)**2)/sig**2)

def vel_ini(x):
  return np.zeros(len(x))

def solucion_ecuacion_onda_1D(dx,t_max,L,c,l0,lL):
  dt = dx / (1.5*c)
  r = (c*dt/dx)**2
  a = 2*(1-r)
  x = np.linspace(0, L, num = 1+int(np.round((L)/dx)))
  lt = 1+int(np.round((t_max)/dt))
  lx = len(x)
  U = np.zeros([lt,lx])
  U[0,:] = pos_ini(x)
  v0 = vel_ini(x)
  U[:,0] = l0
  U[:, -1] = lL
  for i in range(1,lx-1):
    U[1,i] = 0.5*a*U[0,i] + 0.5*r*(U[0,i-1] + U[0,i+1]) + dt*v0[i]

  for n in range(1,lt-1): # time
    for i in range(1,lx-1): # space
      U[n+1,i] = a*U[n,i] + r*(U[n,i-1] + U[n,i+1]) - U[n-1,i]
  return x, U

start_time = time.time()
x, U = solucion_ecuacion_onda_1D(dx,t_max,L,c,l0,lL)
print("Tiempo usado en calcular la solución(s):",time.time() - start_time)

print('x.shape:',x.shape, ' U.shape:',U.shape)

Tiempo usado en calcular la solución(s): 0.019616127014160156
x.shape: (51,)  U.shape: (204, 51)


In [ ]:
def update(num_frame, U, line):
  line.set_ydata(U[num_frame,:])  # update the data.
  return line,

def crear_animacion(x, U, name):
  N = U.shape[0]
  fig = plt.figure(figsize = (9,5), dpi=75)
  ax = plt.gca()
  ax.set_xlim([0,L])
  ax.set_ylim([np.min(U)-1,np.max(U)+1])

  line, = ax.plot(x,U[0],'r')
  plt.tight_layout()
  ani = animation.FuncAnimation(fig, update, N, fargs=(U,line))
  ani.save(name,fps=30)
  plt.close(fig)

start_time = time.time()
crear_animacion(x, U, 'ecuacion_onda_1D.mp4')
print("Tiempo usado en crear la animación (s):",time.time() - start_time)


Tiempo usado en crear la animación (s): 10.11586332321167


In [ ]:
# Mostrar video
from IPython.display import HTML
from base64 import b64encode

name = 'ecuacion_onda_1D.mp4'
mp4 = open(name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=675 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Ecuación de onda 1D (extremos libres)

En este caso los extremos no permanecen en un valor fijo, hay varias formas de tratar estas fronteras, una de ellas es la siguiente:
\begin{equation}
u(t+\Delta t, 0) = u(t+\Delta t, \Delta x)
\end{equation}
\begin{equation}
u(t+\Delta t, l_L) = u(t+\Delta t, l_L-\Delta x)
\end{equation}
Es decir.
\begin{equation}
u^{n+1}_0 = u^{n+1}_1
\end{equation}
\begin{equation}
u^{n+1}_{l_L} = u^{n+1}_{l_L-1}
\end{equation}
Donde se debe incrementar en 2 el tamaño de la malla, creando así un par de nodos "fantasma" que permiten computar cada extremo libre.
Para ser consistentes, se debe modificar la forma en la que se discretiza el espacio, en el código debe cambiar

```python
x = np.linspace(0, L, num = 1+int(np.round((L)/dx)))
```
por esto 

```python
x = np.linspace(-dx, L+dx, num = 1+int(np.round((L + 2*dx)/dx)))
```

Una vez calculados los valores de $x$ y $U$, el retorno de la función debe omitir los valores asociados a los nodos fantasma, en este caso el valor del primer y último nodo para todo $t$.

In [ ]:
dx = 0.05  
L = 1.0
t_max = 3
c = 1.8

#b  = 1./32.  #dt2/dx2
#d  = 1.
#c = d*b

def pos_ini(x):
  #return 4*np.cos(5*np.pi*x)
  sig = 0.1
  return np.exp(-((x-0.5)**2)/sig**2)/np.sqrt(sig)

def vel_ini(x):
  return np.zeros(len(x))

def solucion_ecu_onda_1D_extremos_libres(dx,t_max,L,c):
  dt = dx/ (360*c)
  r = (c*dt/dx)**2
  a = 2*(1-r)
  x = np.linspace(-dx, L+dx, num = 1+int(np.round((L + 2*dx)/dx)))
  v0 = vel_ini(x)
  lt = 1+int(np.round((t_max)/dt))
  lx = len(x) 
  U = np.zeros([lt,lx])
  U[0,:] = pos_ini(x)

  for i in range(1,lx-1):
    U[1,i] = 0.5*a*U[0,i] + 0.5*r*(U[0,i-1] + U[0,i+1]) + dt*v0[i]

  for n in range(1,lt-1): # tiempo
    for i in range(1,lx-1): # espacio
      U[n+1,i] = a*U[n,i] + r*(U[n,i-1] + U[n,i+1]) - U[n-1,i]
    U[n+1,0] = U[n+1,1]
    U[n+1,-1] = U[n+1,-2]

  return x[1:-1], U[:,1:-1]

start_time = time.time()
x, U = solucion_ecu_onda_1D_extremos_libres(dx,t_max,L,c)
print("Tiempo usado en calcular la solución(s):",time.time() - start_time)

print('U.shape:',U.shape)
U = U[::100]
print('U.shape:',U.shape)

Tiempo usado en calcular la solución(s): 1.7300753593444824
U.shape: (38881, 21)
U.shape: (389, 21)


In [ ]:
def update(num_frame, x, U, line, ax):
  y = U[num_frame,:]
  ax.collections.clear()
  piso = np.ones(len(x))*(-7)
  ax.fill_between(x, y, piso, piso<y, color='dodgerblue')
  line.set_ydata(y)  # update the data.
  return line,

def crear_animacion(x, U, name):
  N = U.shape[0]
  fig = plt.figure(figsize = (9,5), dpi = 75)
  ax = plt.gca()
  ax.set_ylim([-7, 7])
  ax.set_xlim([0, L])
  line, = ax.plot(x,U[0])
  plt.tight_layout()
  ani = animation.FuncAnimation(fig, update, N,fargs=(x,U,line, ax))
  ani.save(name,fps=24)
  plt.close(fig)

start_time = time.time()
crear_animacion(x, U, 'ecuacion_onda_1D_extremos_libres.mp4')
print("Tiempo usado en crear la animación (s):", time.time() - start_time)

Tiempo usado en crear la animación (s): 19.24033546447754


In [ ]:
# Mostrar video
from IPython.display import HTML
from base64 import b64encode

name = 'ecuacion_onda_1D_extremos_libres.mp4'
mp4 = open(name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=675 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)